In [ ]:
# gpu monitoring
!nvidia smi

In [1]:
# install
! pip install tensorflow
! pip install transformers

     |████████████████████████████████| 3.1 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 54.1 MB/s 
     |████████████████████████████████| 3.3 MB 45.6 MB/s 
     |████████████████████████████████| 59 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 55.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
# load data from A Modest Proposal
!wget https://raw.githubusercontent.com/vohuy1894/NLP_Group/main/project/book%20text/1080.txt

--2021-11-08 02:33:46--  https://raw.githubusercontent.com/vohuy1894/NLP_Group/main/project/book%20text/1080.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20634 (20K) [text/plain]
Saving to: ‘1080.txt’

1080.txt            100%[===================>]  20.15K  --.-KB/s    in 0.001s  

2021-11-08 02:33:46 (15.0 MB/s) - ‘1080.txt’ saved [20634/20634]



In [4]:
# read the data
text_data = open("1080.txt","r").read()
text_data

'\\r\\n\\r\\n\\r\\n\\r\\n\\r\\nA Modest Proposal\\r\\n\\r\\nFor preventing the children of poor people in Ireland,\\r\\nfrom being a burden on their parents or country,\\r\\nand for making them beneficial to the publick.\\r\\n\\r\\nby Dr. Jonathan Swift\\r\\n\\r\\n1729\\r\\n\\r\\n\\r\\n\\r\\n\\r\\nIt is a melancholy object to those, who walk through this great town,\\r\\nor travel in the country, when they see the streets, the roads, and\\r\\ncabbin-doors crowded with beggars of the female sex, followed by three,\\r\\nfour, or six children, all in rags, and importuning every passenger for\\r\\nan alms. These mothers, instead of being able to work for their honest\\r\\nlivelihood, are forced to employ all their time in stroling to beg\\r\\nsustenance for their helpless infants who, as they grow up, either turn\\r\\nthieves for want of work, or leave their dear native country, to fight\\r\\nfor the Pretender in Spain, or sell themselves to the Barbadoes.\\r\\n\\r\\nI think it is agreed b

In [14]:
# cut down some of text data
text_data = text_data[:500]
text_data

'\\r\\n\\r\\n\\r\\n\\r\\n\\r\\nA Modest Proposal\\r\\n\\r\\nFor preventing the children of poor people in Ireland,\\r\\nfrom being a burden on their parents or country,\\r\\nand for making them beneficial to the publick.\\r\\n\\r\\nby Dr. Jonathan Swift\\r\\n\\r\\n1729\\r\\n\\r\\n\\r\\n\\r\\n\\r\\nIt is a melancholy object to those, who walk through this great town,\\r\\nor travel in the country, when they see the streets, the roads, and\\r\\ncabbin-doors crowded with beggars of the female sex, followed by three,\\r\\nfour, or six childr'

In [15]:
# encode text data as context
input_ids = tokenizer.encode(text_data, return_tensors='tf')

# PyTorch and GPT2

In [11]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [25]:
# generate text until output length
# greedy search for text generation
greedy_output = model.generate(input_ids, max_length=200)

In [26]:
# output
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

\r\n\r\n\r\n\r\n\r\nA Modest Proposal\r\n\r\nFor preventing the children of poor people in Ireland,\r\nfrom being a burden on their parents or country,\r\nand for making them beneficial to the publick.\r\n\r\nby Dr. Jonathan Swift\r\n\r\n1729\r\n\r\n\r\n\r\n\r\nIt is a melancholy object to those, who walk through this great town,\r\nor travel in the country, when they see the streets, the roads, and\r\ncabbin-doors crowded with beggars of the female sex, followed by three,\r\nfour, or six childriders,\r\nfive, or six children,\r\nsix, or seven children


In [27]:
# using beam search and early stopping
beam_output = model.generate(input_ids, max_length=200, num_beams=5, early_stopping=True)

In [28]:
# output
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

\r\n\r\n\r\n\r\n\r\nA Modest Proposal\r\n\r\nFor preventing the children of poor people in Ireland,\r\nfrom being a burden on their parents or country,\r\nand for making them beneficial to the publick.\r\n\r\nby Dr. Jonathan Swift\r\n\r\n1729\r\n\r\n\r\n\r\n\r\nIt is a melancholy object to those, who walk through this great town,\r\nor travel in the country, when they see the streets, the roads, and\r\ncabbin-doors crowded with beggars of the female sex, followed by three,\r\nfour, or six childriders.\r\n\r\n\r\n\r\n\r\


# HuggingFace's Trainer Class (API) & BERT

In [ ]:
# the Trainer Class provides an API for feature-complete training for most use cases
# provides basic training loop but allows for customization through arguments

In [ ]:
# use pre-trained BERT model
from transformers import AutoModel
model = AutoModel.from_pretrained('bert-base-cased')

In [ ]:
# set training arguments/hyperparameters
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [ ]:
# instantiate a Trainer class
from transformers import Trainer
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [ ]:
# train the model
trainer.train()

In [ ]:
# helper function for model evalutation
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# evaluate the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

# References:


* https://huggingface.co/blog/how-to-generate
* https://huggingface.co/transformers/training.html